In [15]:
import pandas as pd

In [16]:
df=pd.read_csv("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.csv")
df

,Dish,Hour,Requirement,Prepare,Inventory,Waste,Unfilled,Sold
0,Samosa,5,50,50.0,60.0,0.0,0.0,50.0
1,Samosa,6,45,50.0,60.0,5.0,0.0,45.0
2,Samosa,7,25,20.0,60.0,0.0,0.0,25.0
3,Samosa,8,20,20.0,55.0,5.0,0.0,20.0
4,Samosa,9,36,30.0,50.0,5.0,1.0,35.0
...,...,...,...,...,...,...,...,...
175,ice cream,9,198,200.0,40.0,4.0,0.0,198.0
176,ice cream,10,218,220.0,38.0,8.0,0.0,218.0
177,ice cream,11,210,200.0,32.0,2.0,0.0,210.0
178,ice cream,12,190,180.0,20.0,0.0,0.0,190.0
